In [2]:
import requests
from bs4 import BeautifulSoup as bs
from pathlib import Path
from urllib.parse import quote

In [4]:
# Open txt file with the required arts
with open('artlist.txt', 'r', encoding='utf-8') as f:
    artlist = [line.strip() for line in f]

# Creating a New Folder
folder = Path.home() / 'Desktop' / 'WikiArt101'
folder.mkdir(exist_ok=True)

# wiki API
api_url = "https://en.wikipedia.org/w/api.php"
se = requests.Session()
# Send a GET request with a valid User-Agent
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

In [4]:
for art in artlist:
    para1 = {"action": "query", "format": "json", "list": "search", "srsearch": art}
    res = se.get(url = api_url, params = para1).json()

    if 'query' in res and res['query']['search']:
        name = res['query']['search'][0]['title']
        
        # request the original full-size image
        para2 = {"action": "query", "format": "json", "prop": "pageimages", 
                "piprop": "original", "titles": name, 
                "formatversion": 2}
        
        res2 = requests.get(api_url, params=para2).json()
        pages = res2.get("query", {}).get("pages", [])
        
        if pages and "original" in pages[0]:
            img_url = pages[0]["original"]["source"]

        if img_url:
            res3 = requests.get(img_url, headers=headers)
            res3.raise_for_status()
    
        filename = art + ".jpg"
        
        with open(folder / filename, "wb") as f:
            f.write(res3.content)

In [7]:
folder_path = Path.home() / "Desktop" / "WikiArt101"
filenames = [file.stem for file in folder_path.iterdir() if file.is_file()]

In [8]:
for art in artlist:
    if art not in filenames:
        print(art)